Data una tabella di grandezza $n$ voglio trovare tutti gli elementi che si ripetono almeno $\frac{n}{k}$ volte. Questa è un'implementazione naïve che non utilizza hashing universale.

In [42]:
import random

In [43]:
T = [ random.randrange(50) for x in range(1000) ]

---

Esempio con $k = 2$, algoritmo di Boyer-Moore

In [44]:
def boyermoore(T):
    c = 0
    v = 0
    for i in T:
        if c == 0:
            v = i
            c += 1
        elif i == v:
            c += 1
        else:
            c -= 1
    return v

---

Con $k > 2$ la soluzione precedente non è più utilizzabile. Innanizitutto creo la struttura count-min sketch, questa implementazione non fa uso di una famiglia universale di funzioni hash (nel futuro la implementerò).

In [54]:
class CMS:
    def __init__(self, l, b):
        self.b = b
        self.M = [ [ 0 for j in range(b) ] for i in range(l) ]
        
    def inc(self, x):
        for r in self.M:
            r[hash(x) % self.b] += 1

    def count(self, x):
        return min([ r[hash(x) % self.b] for r in self.M ])

Ora eseguo `inc` su ogni elemento della mia tabella e con `count` posso avere un'approssimazione del numero di volte in cui appare

In [72]:
M = CMS(10, 40) # 10 tabelle da 40 elementi => 400 righe (|T| = 1000)
for x in T: M.inc(x)
cms_count = { x: M.count(x) for x in T }
cms_count

{19: 21,
 6: 51,
 24: 26,
 32: 19,
 26: 28,
 29: 25,
 42: 38,
 4: 37,
 27: 17,
 30: 18,
 34: 14,
 44: 37,
 7: 48,
 14: 24,
 31: 26,
 1: 40,
 5: 44,
 39: 16,
 49: 45,
 40: 36,
 20: 11,
 43: 46,
 28: 17,
 38: 18,
 2: 38,
 41: 40,
 3: 46,
 48: 29,
 9: 45,
 22: 17,
 17: 15,
 33: 17,
 37: 11,
 21: 22,
 45: 44,
 10: 21,
 13: 22,
 15: 22,
 11: 25,
 16: 16,
 35: 17,
 18: 23,
 8: 29,
 25: 21,
 23: 14,
 36: 26,
 12: 17,
 47: 48,
 46: 51,
 0: 36}

Verifico i risultati

In [68]:
count = dict()
for x in T:
    count[x] = count.setdefault(x, 0) + 1
count

{19: 21,
 6: 22,
 24: 26,
 32: 19,
 26: 28,
 29: 25,
 42: 20,
 4: 19,
 27: 17,
 30: 18,
 34: 14,
 44: 18,
 7: 26,
 14: 24,
 31: 26,
 1: 24,
 5: 23,
 39: 16,
 49: 22,
 40: 18,
 20: 11,
 43: 22,
 28: 17,
 38: 18,
 2: 18,
 41: 16,
 3: 24,
 48: 16,
 9: 23,
 22: 17,
 17: 15,
 33: 17,
 37: 11,
 21: 22,
 45: 21,
 10: 21,
 13: 22,
 15: 22,
 11: 25,
 16: 16,
 35: 17,
 18: 23,
 8: 13,
 25: 21,
 23: 14,
 36: 26,
 12: 17,
 47: 22,
 46: 29,
 0: 18}

Quanto distano i due risultati? Calcolo l'errore medio

In [71]:
sum([ abs(cms_count[x] - count[x]) for x in count ]) / len(count)

8.28